In [ ]:
import pandas as pd
import numpy as np

import os

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import log_loss

from IPython.display import display

%matplotlib inline

# Limitations
- In the interest of time and simplicity we will ignore the players and events as part of this year's model. It could be expanded on at a later time to enchance the model with this additional data.
- Seeds will not be used as they are a biased ranking

In [ ]:
data_dir = 'data_2018'

In [ ]:
df_teams = pd.read_csv(os.path.join(data_dir, 'Teams.csv'))
df_teams.head()

In [ ]:
df_teams['D1Seasons'] = df_teams['LastD1Season'] - df_teams['FirstD1Season']
df_teams.head()

In [ ]:
df_detailed = pd.read_csv(os.path.join(data_dir, 'RegularSeasonDetailedResults.csv'))
df_detailed.head()

In [ ]:
df_coaches = pd.read_csv(os.path.join(data_dir, 'TeamCoaches.csv'))
df_coaches.head()

In [ ]:
df_coaches = df_coaches.sort_values(by=['CoachName', 'Season'])
df_coaches['NumDaysCoaching'] = df_coaches['LastDayNum'] - df_coaches['FirstDayNum']
df_coaches['TotalDaysCoaching'] = df_coaches.groupby('CoachName')['NumDaysCoaching'].cumsum()

# Which iteration the coach as been with the team
# Team A -> Team B -> Team A = 1 -> 2 -> 3
df_coaches['TeamNumber'] = df_coaches['TeamID'].ne(df_coaches['TeamID'].shift().bfill()).astype(int)
df_coaches['TeamNumber'] = df_coaches.groupby(['CoachName'])['TeamNumber'].cumsum()

df_coaches['TeamTenure'] = df_coaches.groupby(['CoachName', 'TeamNumber'])['NumDaysCoaching'].cumsum()
df_coaches.sample(20)

In [ ]:
def calc_gameid(row):
    min_id = min(row['WTeamID'], row['LTeamID'])
    max_id = max(row['WTeamID'], row['LTeamID'])
    season = row['Season']
    return f'{season}_{min_id}_{max_id}'

In [ ]:
df_detailed['GameID'] = df_detailed.apply(calc_gameid, axis='columns')

In [ ]:
df_detailed_team_W = pd.merge(left = df_detailed,
        right = df_teams,
        left_on = ['WTeamID'],
        right_on = ['TeamID'])

df_detailed_team_L = pd.merge(left = df_detailed_team_W,
        right = df_teams,
        left_on = ['LTeamID'],
        right_on = ['TeamID'],
        suffixes = ('_W', '_L'))

df_master = pd.merge(left = df_detailed_team_W,
         right = df_detailed_team_L,
         on = list(df_detailed.columns),
         suffixes = ('_W', '_L')
        )

df_master = pd.merge(
    left = df_master,
    right = df_coaches,
    left_on = ['Season', 'WTeamID'],
    right_on = ['Season', 'TeamID']#,
    #suffixes = ('_W', '_L')
)

df_master = pd.merge(
    left = df_master,
    right = df_coaches,
    left_on = ['Season', 'LTeamID'],
    right_on = ['Season', 'TeamID'],
     suffixes = ('_W', '_L')
)
df_master

In [ ]:
df_master['GameID'].value_counts()

Filter out coaches based on the game day to handle coaches who are replaced mid-season

In [ ]:
mask = (df_master['FirstDayNum_L'] <= df_master['DayNum'])
mask = mask & (df_master['LastDayNum_L'] >= df_master['DayNum'])
mask = mask & (df_master['FirstDayNum_W'] <= df_master['DayNum'])
mask = mask & (df_master['LastDayNum_W'] >= df_master['DayNum'])
df_master.loc[mask, 'GameID'].value_counts()


In [ ]:
df_master = df_master.loc[mask, :]

In [ ]:
df_master.sample(20)